In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# BA885- Advanced Analytics- Applied Optimization- Team 2

## Problem Statement

In urban areas, bike-sharing systems have witnessed growing demand as a sustainable mode of transportation. The primary challenge facing these systems is the effective allocation of two types of bikes – standard and electric – across different stations to maximize profitability. Our analysis is driven by the objective of determining the optimal number of each type of bike to station at our chosen 10 Citibike stations in NYC. This includes addressing the disequilibrium between demand and supply during various periods, which places significant pressure on operational repositioning efforts. By optimizing bike allocation, we aim to enhance operational efficiency and financial sustainability, considering both operational costs and potential revenues.

## Methodology

- Data Collection:
To address our optimization problem, we combined datasets from Citibikes, nyc.gov, and Google. The dataset encompasses two pivotal components: New York Citibike station data and Citibikes trips data. The former includes information on station capacity, empty docks, available bikes, and disabled bikes, while the latter provides detailed insights into each customer's bike trip.

- Data Preparation:
Ensuring the accuracy and completeness of our data by filling gaps, and integrating different datasets into a unified format. Using statistical and visual tools to identify bike usage trends, patterns, and outliers in the data.

- Model 1:
Our approach involves two models to tackle the optimization problem. The first model is an assignment optimization model aimed at determining optimal demand numbers for revenue maximization at selected stations during the morning slot from 8 a.m. to 10 a.m.
Feeding Values into Model 2:
The optimal demand numbers derived from Model 1 guide us in understanding the morning bike requirements at each station, a critical input for our second optimization model.

- Model 2:
Our second model is a max flow optimization problem. Here, we seek to identify the optimal path for moving bikes overnight from one station to another at minimal cost. Each station can either supply bikes or may require additional bikes to meet the optimal demand in the morning.
By structuring our methodology in this manner, we aim to streamline the optimization process, maximizing the efficiency of Citibike operations while minimizing associated costs.


## Reason for Undertaking

This endeavor aims to streamline resource utilization, reduce wastage, and ultimately enhance profitability by aligning bike supply with user demand. The successful execution of this project has the potential to yield several substantial benefits, including:


- Enhanced Customer Satisfaction: By ensuring that each station has an adequate number of bikes to meet demand, we aim to provide a superior experience to our users, minimizing unmet requests and enhancing overall satisfaction.
- Asset Optimization: Efficient allocation of bikes translates to optimized asset utilization. This approach minimizes the financial impact of idle or underutilized bikes while maximizing their use.
- Improved Financial Performance: An optimized allocation strategy is expected to positively impact the company's financial performance. By reducing operational costs and enhancing revenues, the company gains financially.

The outcomes of this model will determine the ideal number of bikes to be docked at each of the selected stations. The responsibility of implementing these recommendations will be assigned to designated personnel, called 'bike angels.'


## Data Summary

In [2]:
df = pd.read_csv("/Users/ishan/Downloads/202310-citibike-tripdata.csv",parse_dates=['started_at','ended_at'],low_memory=False)

In [3]:
print(df.info())
df.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3823673 entries, 0 to 3823672
Data columns (total 13 columns):
 #   Column              Dtype         
---  ------              -----         
 0   ride_id             object        
 1   rideable_type       object        
 2   started_at          datetime64[ns]
 3   ended_at            datetime64[ns]
 4   start_station_name  object        
 5   start_station_id    object        
 6   end_station_name    object        
 7   end_station_id      object        
 8   start_lat           float64       
 9   start_lng           float64       
 10  end_lat             float64       
 11  end_lng             float64       
 12  member_casual       object        
dtypes: datetime64[ns](2), float64(4), object(7)
memory usage: 379.2+ MB
None


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,F01D2D54E9E60D6E,classic_bike,2023-10-03 02:48:38,2023-10-03 02:48:40,Columbus Pl & Atlantic Ave,4146.02,Columbus Pl & Atlantic Ave,4146.02,40.677223,-73.922792,40.677170,-73.922850,casual
1,CAE4EDBEA07001BD,classic_bike,2023-10-11 16:03:17,2023-10-11 16:45:26,Central Park West & W 85 St,7354.01,Central Park West & W 85 St,7354.01,40.784760,-73.969862,40.784760,-73.969862,casual
2,FDC34BAD31193E07,classic_bike,2023-10-11 19:57:13,2023-10-11 20:20:10,Hicks St & Montague St,4645.09,5 St & 6 Ave,3874.01,40.694974,-73.995936,40.670484,-73.982090,casual
3,DFEA5E65AE91CE2A,classic_bike,2023-10-10 20:18:22,2023-10-10 20:18:37,Atlantic Ave & Furman St,4614.04,Atlantic Ave & Furman St,4614.04,40.691669,-74.000139,40.691652,-73.999979,casual
4,48299D8BE9B55255,classic_bike,2023-10-17 16:26:58,2023-10-17 16:34:27,E 41 St & Madison Ave (SE corner),6432.10,E 58 St & 3 Ave,6762.02,40.751845,-73.979585,40.760958,-73.967245,casual


In [4]:
from tabulate import tabulate
pd.set_option('display.width', 10000)

# Summary of numerical columns
numerical_summary = df.describe()

# Summary of categorical columns
categorical_summary = df.describe(include='object')

# Print formatted numerical columns summary
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns
print(numerical_cols)
print("Numerical Columns Summary:\n")
print(tabulate(numerical_summary, headers='keys', tablefmt='grid'))

# Print formatted categorical columns summary
categorical_cols = df.select_dtypes(include=['object', 'category']).columns
print(categorical_cols)
print("\nCategorical Columns Summary:\n")
print(tabulate(categorical_summary, headers='keys', tablefmt='grid'))

Index(['start_lat', 'start_lng', 'end_lat', 'end_lng'], dtype='object')
Numerical Columns Summary:

+-------+-------------------------------+-------------------------------+--------------+---------------+--------------+---------------+
|       | started_at                    | ended_at                      |    start_lat |     start_lng |      end_lat |       end_lng |
+=======+===============================+===============================+==============+===============+==============+===============+
| count | 3823673                       | 3823673                       |  3.82367e+06 |   3.82367e+06 |  3.82129e+06 |   3.82129e+06 |
+-------+-------------------------------+-------------------------------+--------------+---------------+--------------+---------------+
| mean  | 2023-10-16 06:55:04.447225600 | 2023-10-16 07:09:51.799132928 | 40.7387      | -73.9727      | 40.7385      | -73.9728      |
+-------+-------------------------------+-------------------------------+-----------

In [5]:
df.nunique().sort_values(ascending=False)

ride_id               3823673
ended_at              1666063
started_at            1661539
start_lat              883455
start_lng              728676
end_station_name         2151
end_lat                  2124
end_lng                  2124
start_station_name       2116
end_station_id           2098
start_station_id         2063
rideable_type               2
member_casual               2
dtype: int64

In [6]:
missing_percentage = df.isnull().mean() * 100
missing_percentage_sorted = missing_percentage.sort_values(ascending=False)
print(missing_percentage_sorted.head(5))

end_station_name      0.392764
end_station_id        0.392764
start_station_name    0.151320
start_station_id      0.151320
end_lat               0.062427
dtype: float64


## Data Preparation

### **Identifying the most popular stations for Resource Allocation** :
&nbsp;&nbsp;&nbsp;&nbsp;This segment is dedicated to pinpointing the pivotal hubs within our bike-sharing network. We concentrate on the influx of departures during peak urban transit times, specifically from 8 am to 10 am. This window captures the quintessential rush hour, providing us with a rich dataset to analyze user patterns.

Our methodology entails a thorough examination of trip frequencies across all stations during the specified morning rush period. We’ll then distill this data into a daily average, representing the typical demand throughout October 2023. This approach ensures that our demand figures are not skewed by outliers or one-off events but are instead reflective of consistent user behavior.

To ensure city-wide service optimization, our selection criteria for top-tier stations will hinge on two pivotal factors: the average demand and strategic geographic distribution. By doing so, we aim to encompass a wide array of urban territories, guaranteeing that our resources are not just concentrated in high-traffic areas but are also accessible to a broader demographic.

Further, we will delve into the duration aspect, calculating the average trip length for bicycles commencing their journey from these high-demand stations. This metric will offer valuable insights into the temporal patterns of our users, enabling us to fine-tune our deployment strategy and enhance overall service efficiency.

In [22]:
top_stations = df['start_station_name'].value_counts().head(30).index
top_stations

Index(['W 21 St & 6 Ave', 'West St & Chambers St', 'E 41 St & Madison Ave (SE corner)', 'University Pl & E 14 St', 'Broadway & W 58 St', '11 Ave & W 41 St', 'Ave A & E 14 St', '1 Ave & E 68 St', 'Broadway & W 25 St', '7 Ave & Central Park South', '6 Ave & W 33 St', '8 Ave & W 31 St', 'W 31 St & 7 Ave', 'E 17 St & Broadway', 'E 33 St & 1 Ave', 'W 41 St & 8 Ave', '12 Ave & W 40 St', 'Central Park S & 6 Ave', 'W 30 St & 10 Ave', 'Cooper Square & Astor Pl', '2 Ave & E 29 St', 'W 20 St & 10 Ave', 'West St & Liberty St', 'W 22 St & 10 Ave', 'Broadway & E 14 St', '4 Ave & E 12 St', 'Cleveland Pl & Spring St', 'W 13 St & 5 Ave', '6 Ave & W 34 St', '8 Ave & W 33 St'], dtype='object', name='start_station_name')

In [23]:
df_filtered = df[df['start_station_name'].isin(top_stations) & 
                 df['started_at'].dt.hour.isin([8,9])]

In [24]:
avg_trips_per_day = df_filtered.groupby('start_station_name').size() / \
      df_filtered['started_at'].dt.date.nunique()

In [25]:
print(avg_trips_per_day.sort_values(ascending=False))


start_station_name
E 41 St & Madison Ave (SE corner)    56.838710
1 Ave & E 68 St                      50.354839
W 21 St & 6 Ave                      50.225806
E 33 St & 1 Ave                      46.677419
12 Ave & W 40 St                     46.516129
Broadway & W 58 St                   46.419355
8 Ave & W 31 St                      45.935484
11 Ave & W 41 St                     45.193548
2 Ave & E 29 St                      41.709677
W 20 St & 10 Ave                     40.064516
West St & Chambers St                39.290323
W 22 St & 10 Ave                     38.387097
W 30 St & 10 Ave                     38.064516
West St & Liberty St                 37.258065
Ave A & E 14 St                      37.032258
W 31 St & 7 Ave                      36.387097
6 Ave & W 33 St                      34.806452
6 Ave & W 34 St                      34.548387
University Pl & E 14 St              34.225806
8 Ave & W 33 St                      33.387097
E 17 St & Broadway                   33.1

In [26]:
df_filtered['trip_duration'] = (df_filtered['ended_at'] - df_filtered['started_at']).dt.total_seconds() / 60  # duration in minutes
avg_trip_time = df_filtered.groupby('start_station_name')['trip_duration'].mean()


/var/folders/gj/nnzd_bq105s65r5x_8lb4m7c0000gn/T/ipykernel_36400/3513231363.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['trip_duration'] = (df_filtered['ended_at'] - df_filtered['started_at']).dt.total_seconds() / 60  # duration in minutes


In [27]:
avg_trip_time.sort_values(ascending=False)

start_station_name
Central Park S & 6 Ave               25.519549
12 Ave & W 40 St                     19.035922
7 Ave & Central Park South           18.317988
11 Ave & W 41 St                     15.555068
6 Ave & W 34 St                      15.000778
West St & Chambers St                14.435276
Broadway & W 58 St                   14.312868
8 Ave & W 31 St                      14.308684
W 41 St & 8 Ave                      13.607115
W 20 St & 10 Ave                     13.341613
West St & Liberty St                 13.234069
1 Ave & E 68 St                      13.171695
8 Ave & W 33 St                      13.155668
W 21 St & 6 Ave                      13.006605
E 33 St & 1 Ave                      12.726422
W 22 St & 10 Ave                     12.529328
University Pl & E 14 St              12.409881
W 31 St & 7 Ave                      12.332033
Cooper Square & Astor Pl             12.310421
E 41 St & Madison Ave (SE corner)    12.306139
Broadway & W 25 St                   11.5

### **Comprehensive Overview of Top 10 Stations by Daily Average Departures and Trip Duration**

In [28]:
# Combine the two Series into a DataFrame
combined_df = pd.concat([avg_trips_per_day, avg_trip_time], axis=1)

# Rename the columns for clarity
combined_df.columns = ['Average Trips Per Day', 'Average Trip Duration (Minutes)']

# Optionally, you can sort by one of the columns
combined_df.sort_values(by='Average Trips Per Day', ascending=False, inplace=True)

locations = [
    "E 17 St & Broadway", "W 21 St & 6 Ave", "West St & Chambers St",
    "7 Ave & Central Park South", "8 Ave & W 31 St", "Central Park S & 6 Ave",
    "University Pl & E 14 St", "W 20 St & 10 Ave", "E 41 St & Madison Ave (SE corner)",
    "West St & Liberty St"
]
filtered_combined_df = combined_df.loc[locations]

# Display the combined table
print(filtered_combined_df)

filtered_combined_df.to_csv('stations_demand_avg_time.csv', index=True)


                                   Average Trips Per Day  Average Trip Duration (Minutes)
start_station_name                                                                       
E 17 St & Broadway                             33.193548                        11.204033
W 21 St & 6 Ave                                50.225806                        13.006605
West St & Chambers St                          39.290323                        14.435276
7 Ave & Central Park South                     32.548387                        18.317988
8 Ave & W 31 St                                45.935484                        14.308684
Central Park S & 6 Ave                         21.451613                        25.519549
University Pl & E 14 St                        34.225806                        12.409881
W 20 St & 10 Ave                               40.064516                        13.341613
E 41 St & Madison Ave (SE corner)              56.838710                        12.306139
West St & 

### **Interactive Visualization of our selected Citibike Station Network Across New York City**

In [31]:
import folium
from folium.features import DivIcon

# Latitude and Longitude for the updated list of stations
stations = {
    "E 17 St & Broadway": {"lat": 40.737050, "lon": -73.990093},
    "W 21 St & 6 Ave": {"lat": 40.741740, "lon": -73.994156},
    "West St & Chambers St": {"lat": 40.717548, "lon": -74.013221},
    "7 Ave & Central Park South": {"lat": 40.766368, "lon": -73.977688},
    "8 Ave & W 31 St": {"lat": 40.750020, "lon": -73.994760},
    "Central Park S & 6 Ave": {"lat": 40.765909, "lon": -73.976342},
    "University Pl & E 14 St": {"lat": 40.734927, "lon": -73.992005},
    "W 20 St & 10 Ave": {"lat": 40.746745, "lon": -74.007756},
    "E 41 St & Madison Ave (SE corner)": {"lat": 40.752722, "lon": -73.977987},
    "West St & Liberty St": {"lat": 40.711444, "lon": -74.014847}
}

# Create a map
m = folium.Map(location=[40.741895, -73.989308], zoom_start=13)

# Add markers with station names
for station, coord in stations.items():
    folium.Marker([coord['lat'], coord['lon']]).add_to(m)
    folium.map.Marker(
        [coord['lat'], coord['lon']],
        icon=DivIcon(
            icon_size=(150,36),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 10pt">{station}</div>',
            )
        ).add_to(m)

# Display the map
m


In [21]:
import pandas as pd
import numpy as np
from math import radians, cos, sin, asin, sqrt

# Haversine formula to calculate the distance between two lat/lon points
def haversine(lon1, lat1, lon2, lat2):
    # Convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    r = 6371  # Radius of Earth in kilometers
    return c * r

# Stations with their latitude and longitude
stations = {
    "E 17 St & Broadway": {"lat": 40.737050, "lon": -73.990093},
    "W 21 St & 6 Ave": {"lat": 40.741740, "lon": -73.994156},
    "West St & Chambers St": {"lat": 40.717548, "lon": -74.013221},
    "7 Ave & Central Park South": {"lat": 40.766368, "lon": -73.977688},
    "8 Ave & W 31 St": {"lat": 40.750020, "lon": -73.994760},
    "Central Park S & 6 Ave": {"lat": 40.765909, "lon": -73.976342},
    "University Pl & E 14 St": {"lat": 40.734927, "lon": -73.992005},
    "W 20 St & 10 Ave": {"lat": 40.746745, "lon": -74.007756},
    "E 41 St & Madison Ave (SE corner)": {"lat": 40.752722, "lon": -73.977987},
    "West St & Liberty St": {"lat": 40.711444, "lon": -74.014847}
}


# Average biking speed in km/h
average_speed_kmh = 15

# List to store the data
data = []

# Calculate distances and estimated bike time for each station pair
for from_station, from_coords in stations.items():
    for to_station, to_coords in stations.items():
        if from_station != to_station:
            distance = haversine(from_coords['lon'], from_coords['lat'], to_coords['lon'], to_coords['lat'])
            time_minutes = (distance / average_speed_kmh) * 60  # Convert hours to minutes
            data.append({'From': from_station, 'To': to_station, 'Distance (km)': distance, 'Estimated Time (min)': time_minutes})

# Create DataFrame from the list
distances_df = pd.DataFrame(data)

# Display the DataFrame
print(distances_df)

distances_df.to_csv('stations_time.csv', index=True)



                    From                                 To  Distance (km)  Estimated Time (min)
0     E 17 St & Broadway                    W 21 St & 6 Ave       0.623814              2.495255
1     E 17 St & Broadway              West St & Chambers St       2.915602             11.662407
2     E 17 St & Broadway         7 Ave & Central Park South       3.423387             13.693548
3     E 17 St & Broadway                    8 Ave & W 31 St       1.494832              5.979326
4     E 17 St & Broadway             Central Park S & 6 Ave       3.411631             13.646525
..                   ...                                ...            ...                   ...
85  West St & Liberty St                    8 Ave & W 31 St       4.611313             18.445253
86  West St & Liberty St             Central Park S & 6 Ave       6.870386             27.481545
87  West St & Liberty St            University Pl & E 14 St       3.244018             12.976074
88  West St & Liberty St      

In [29]:
distances_df.describe()

,Distance (km),Estimated Time (min)
count,90.000000,90.000000
mean,2.874364,11.497456
std,1.687520,6.750080
min,0.124316,0.497265
25%,1.517575,6.070300
50%,2.915602,11.662407
75%,3.689055,14.756222
max,6.870386,27.481545


## Model 1: Optimising bike allocation to each station

&nbsp;&nbsp;&nbsp;&nbsp;The primary aim of this model is to strategically allocate bicycles to each station in a manner that aligns with past usage patterns while also maximizing potential revenue. We will leverage historical demand data, station capacity, and revenue metrics to inform our distribution strategy.

### **Objective Function**

&nbsp;&nbsp;&nbsp;&nbsp;Maximize Revenue: Increase the overall profitability by ensuring that bikes are positioned at stations in accordance with anticipated revenue generation.

### **Decision Variables**

&nbsp;&nbsp;&nbsp;&nbsp;Bicycle Allocation: Determine the optimal quantity of bicycles to be stationed at each location.

### **Constraints**


- Bike Availability: The sum total of bikes allocated across all stations must not surpass the overall number of bikes available.

- Demand-Sensitive Ceiling: The allocation for each station should not exceed double the historical demand to prevent over-saturation.

- Operational Capacity: Each station's allocation must respect its net capacity, which accounts for a 10% buffer for maintenance activities and retains a 5% margin for available docks.

- Demand Fulfillment: The allocation at each station must at least meet the historical demand to ensure customer needs are adequately met.

- Non-Negativity: The number of bikes allocated must be a non-negative value, reflecting a realistic and implementable model.

### **Results and Sensitivty Analysis**

&nbsp;&nbsp;&nbsp;&nbsp;This model is designed to optimize resource distribution, catering to user demand while also considering operational constraints, thereby enhancing the efficiency and profitability of the Citibike network.

## Model 2: Optimising the flow of bikes across stations

&nbsp;&nbsp;&nbsp;&nbsp;For the subsequent phase of our strategy, we build upon the allocations determined in Model 1 to orchestrate the distribution of bikes between stations, ensuring that demand is satisfied in the most cost-effective manner

### **Objective Function**

&nbsp;&nbsp;&nbsp;&nbsp;Minimize Transit Costs: Our focus is to reduce the expenses involved in relocating bikes between stations while adhering to the forecasted demand patterns.

### **Decision Variables**

&nbsp;&nbsp;&nbsp;&nbsp;Transit Quantities: We have identified 90 decision variables representing the number of bikes to be transported from each station to another.

### **Constraints**

- Transfer Limits: Each decision variable is capped by a transfer limit, which is contingent upon the transit time between stations. For transit times less than 11.66, the upper limit is 18 bikes; for times exceeding 11.66, the limit is reduced to 12 bikes, reflecting the constraints imposed by longer distances.

- Demand Satisfaction: The sum of bikes distributed from and received at each station, combined with the station's initial bike availability, must equate to the station's projected optimal demand.

- Station Capacity Compliance: The total number of bikes arriving at any station, when added to those already present, must not exceed the station's capacity, considering operational thresholds.

- Non-Negativity: All decision variables must be non-negative to ensure practical and feasible solutions.

### **Results and Sensitivty Analysis**

&nbsp;&nbsp;&nbsp;&nbsp;This model is designed to optimize the logistics of bike redistribution across the network, ensuring that every station is adequately stocked to meet user demand while maintaining operational efficiency and cost control.

## Limitations

&nbsp;&nbsp;&nbsp;&nbsp;**Limitations of Model 1: Strategic Bike Distribution Framework**

- Historical Demand as Predictor: The model assumes that historical demand is a reliable predictor of future usage, which may not account for fluctuations due to seasonal changes, special events, or shifts in population and commuting patterns.

- Static Capacity Constraints: The model does not account for dynamic capacity changes, such as temporary station closures or expansions that can affect the availability of docks and bikes.

- Maintenance and Docks Availability: By reserving a fixed percentage for maintenance and empty docks, the model may either overestimate or underestimate the actual number of bikes and docks needed, as these needs can vary over time.

- Revenue Estimation: The revenue maximization assumes a direct correlation between the number of bikes and revenue, not considering other factors like pricing changes, promotional campaigns, or competitive services.

- Demand Fulfillment: The constraint that the number of bikes at each station must meet historical demand might not be flexible enough to respond to real-time demand spikes or unexpected lulls.

&nbsp;&nbsp;&nbsp;&nbsp;**Limitations of Model 2: Bike Redistribution Network Optimization**

- Transit Time Constraints: The limits based on transit times may not be flexible enough to accommodate varying traffic conditions or different times of the day when redistribution may be faster or slower.

- Static Demand and Supply: The model uses a static picture of demand and supply from Model 1 and does not account for intra-day variations in bike usage patterns.

- Transfer Limitations: The binary cutoff for the transfer limit based on transit time may not reflect the true capacity or efficiency of bike transfer between stations, which could be more variable.

- Cost Calculations: The model assumes that the cost of moving bikes is a linear function of the number of bikes and distance, which might not reflect volume discounts, fixed costs, or economies of scale in bike logistics.

- Operational Practicalities: Practical issues such as the time required for loading and unloading bikes, staff availability, and the synchronization of bike transfers with other operational activities are not considered.

- Single Objective Focus: The focus on minimizing transit costs may overlook other operational objectives, such as minimizing the environmental impact of redistribution or balancing workload among staff.


For both models, the real-world applicability may be limited by the exclusion of unpredictable factors and the assumption of consistent conditions, which rarely hold true in dynamic urban environments. Additionally, both models assume perfect compliance with projected figures, which can diverge from actual outcomes due to numerous unforeseen variables.
